# MAP 536 - Python for Data Science - Predicting Cyclist Traffic in Paris

## Prediction

### adding french holidays

Import all necessary packages

In [134]:

import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import numpy as np
from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import TimeSeriesSplit


1. Load and filter original dataset

In [152]:
# Load training and testing datasets
train_data = pd.read_parquet(Path("data") / "train.parquet")
test_data = pd.read_parquet(Path("data") / "test.parquet")

In [153]:
train_data.head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,counter_technical_id,latitude,longitude,log_bike_count
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,0.000000
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,0.693147
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,0.000000
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,1.609438
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,2.302585


In [154]:
train_data.drop(columns=['counter_id', 'counter_installation_date', 'counter_technical_id', 'site_id'], inplace=True)

# Ensure the 'date' column is in datetime format
train_data['date'] = pd.to_datetime(train_data['date'])

# Extract the hour first, then extract the date
train_data['hour'] = train_data['date'].dt.hour
train_data['date'] = train_data['date'].dt.date
train_data.head()

,counter_name,site_name,bike_count,date,latitude,longitude,log_bike_count,hour
48321,28 boulevard Diderot E-O,28 boulevard Diderot,0.0,2020-09-01,48.846028,2.375429,0.000000,2
48324,28 boulevard Diderot E-O,28 boulevard Diderot,1.0,2020-09-01,48.846028,2.375429,0.693147,3
48327,28 boulevard Diderot E-O,28 boulevard Diderot,0.0,2020-09-01,48.846028,2.375429,0.000000,4
48330,28 boulevard Diderot E-O,28 boulevard Diderot,4.0,2020-09-01,48.846028,2.375429,1.609438,15
48333,28 boulevard Diderot E-O,28 boulevard Diderot,9.0,2020-09-01,48.846028,2.375429,2.302585,18


2. Import new dataset (holidays in France)

In [155]:
holiday_data = pd.read_csv(Path("data") / "jours_feries_metropole.csv")
holiday_data.rename(columns={'date': 'h_date', 'nom_jour_ferie': 'is_holiday'}, inplace=True)
holiday_data['h_date'] = pd.to_datetime(holiday_data['h_date']).dt.date
holiday_data.drop(columns=['zone','annee'], inplace=True) # we drop the redundant or irrelevant columns 
holiday_data.rename(columns={"A": "a", "B": "c"})


holiday_data.head()

,h_date,is_holiday
0,2003-01-01,1er janvier
1,2003-04-21,Lundi de Pâques
2,2003-05-01,1er mai
3,2003-05-08,8 mai
4,2003-05-29,Ascension


3. Merge the two datasets by date

In [156]:
merged_train_data = pd.merge(train_data, holiday_data, left_on='date', right_on='h_date', how='left')
merged_train_data['date'] = pd.to_datetime(merged_train_data['date'])

merged_train_data['is_holiday'] = merged_train_data['is_holiday'].fillna(0)
merged_train_data['is_holiday'] = merged_train_data['is_holiday'].apply(lambda x: 1 if x != 0 else 0)



merged_train_data.head()

,counter_name,site_name,bike_count,date,latitude,longitude,log_bike_count,hour,h_date,is_holiday
0,28 boulevard Diderot E-O,28 boulevard Diderot,0.0,2020-09-01,48.846028,2.375429,0.000000,2,NaN,0
1,28 boulevard Diderot E-O,28 boulevard Diderot,1.0,2020-09-01,48.846028,2.375429,0.693147,3,NaN,0
2,28 boulevard Diderot E-O,28 boulevard Diderot,0.0,2020-09-01,48.846028,2.375429,0.000000,4,NaN,0
3,28 boulevard Diderot E-O,28 boulevard Diderot,4.0,2020-09-01,48.846028,2.375429,1.609438,15,NaN,0
4,28 boulevard Diderot E-O,28 boulevard Diderot,9.0,2020-09-01,48.846028,2.375429,2.302585,18,NaN,0


In [157]:
# check for xmas dates
merged_train_data[(merged_train_data['date'] == '2020-12-25')]


,counter_name,site_name,bike_count,date,latitude,longitude,log_bike_count,hour,h_date,is_holiday
2377,28 boulevard Diderot E-O,28 boulevard Diderot,1.0,2020-12-25,48.846028,2.375429,0.693147,4,2020-12-25,1
2380,28 boulevard Diderot E-O,28 boulevard Diderot,0.0,2020-12-25,48.846028,2.375429,0.000000,6,2020-12-25,1
2383,28 boulevard Diderot E-O,28 boulevard Diderot,0.0,2020-12-25,48.846028,2.375429,0.000000,8,2020-12-25,1
2386,28 boulevard Diderot E-O,28 boulevard Diderot,4.0,2020-12-25,48.846028,2.375429,1.609438,10,2020-12-25,1
2389,28 boulevard Diderot E-O,28 boulevard Diderot,0.0,2020-12-25,48.846028,2.375429,0.000000,13,2020-12-25,1
...,...,...,...,...,...,...,...,...,...,...
450581,254 rue de Vaugirard SO-NE,254 rue de Vaugirard,8.0,2020-12-25,48.839770,2.301980,2.197225,7,2020-12-25,1
450584,254 rue de Vaugirard SO-NE,254 rue de Vaugirard,19.0,2020-12-25,48.839770,2.301980,2.995732,9,2020-12-25,1
450587,254 rue de Vaugirard SO-NE,254 rue de Vaugirard,29.0,2020-12-25,48.839770,2.301980,3.401197,13,2020-12-25,1
450590,254 rue de Vaugirard SO-NE,254 rue de Vaugirard,1.0,2020-12-25,48.839770,2.301980,0.693147,20,2020-12-25,1


In [158]:
# drop redundant date column
merged_train_data.drop(columns=['h_date'], inplace=True)
merged_train_data.head()


,counter_name,site_name,bike_count,date,latitude,longitude,log_bike_count,hour,is_holiday
0,28 boulevard Diderot E-O,28 boulevard Diderot,0.0,2020-09-01,48.846028,2.375429,0.000000,2,0
1,28 boulevard Diderot E-O,28 boulevard Diderot,1.0,2020-09-01,48.846028,2.375429,0.693147,3,0
2,28 boulevard Diderot E-O,28 boulevard Diderot,0.0,2020-09-01,48.846028,2.375429,0.000000,4,0
3,28 boulevard Diderot E-O,28 boulevard Diderot,4.0,2020-09-01,48.846028,2.375429,1.609438,15,0
4,28 boulevard Diderot E-O,28 boulevard Diderot,9.0,2020-09-01,48.846028,2.375429,2.302585,18,0


4. Repeat the process for the testing data

In [159]:
# do the same for test data

# Ensure the 'date' column is in datetime format
test_data['date'] = pd.to_datetime(test_data['date'])

# Extract the hour first, then extract the date
test_data['hour'] = test_data['date'].dt.hour
test_data['date'] = test_data['date'].dt.date

test_data.drop(columns=['counter_id', 'counter_installation_date', 'counter_technical_id', 'site_id'], inplace=True)

test_data.head()

,counter_name,site_name,bike_count,date,latitude,longitude,log_bike_count,hour
56474,28 boulevard Diderot E-O,28 boulevard Diderot,1.0,2021-08-10,48.846028,2.375429,0.693147,5
56477,28 boulevard Diderot E-O,28 boulevard Diderot,2.0,2021-08-10,48.846028,2.375429,1.098612,6
56480,28 boulevard Diderot E-O,28 boulevard Diderot,1.0,2021-08-10,48.846028,2.375429,0.693147,7
56483,28 boulevard Diderot E-O,28 boulevard Diderot,0.0,2021-08-10,48.846028,2.375429,0.000000,9
56486,28 boulevard Diderot E-O,28 boulevard Diderot,1.0,2021-08-10,48.846028,2.375429,0.693147,10


In [160]:
merged_test_data = pd.merge(test_data, holiday_data, left_on='date', right_on='h_date', how='left')
merged_test_data['date'] = pd.to_datetime(merged_test_data['date'])

merged_test_data['is_holiday'] = merged_test_data['is_holiday'].fillna(0)
merged_test_data['is_holiday'] = merged_test_data['is_holiday'].apply(lambda x: 1 if x != 0 else 0)



merged_test_data.head()

,counter_name,site_name,bike_count,date,latitude,longitude,log_bike_count,hour,h_date,is_holiday
0,28 boulevard Diderot E-O,28 boulevard Diderot,1.0,2021-08-10,48.846028,2.375429,0.693147,5,NaN,0
1,28 boulevard Diderot E-O,28 boulevard Diderot,2.0,2021-08-10,48.846028,2.375429,1.098612,6,NaN,0
2,28 boulevard Diderot E-O,28 boulevard Diderot,1.0,2021-08-10,48.846028,2.375429,0.693147,7,NaN,0
3,28 boulevard Diderot E-O,28 boulevard Diderot,0.0,2021-08-10,48.846028,2.375429,0.000000,9,NaN,0
4,28 boulevard Diderot E-O,28 boulevard Diderot,1.0,2021-08-10,48.846028,2.375429,0.693147,10,NaN,0


In [161]:
# drop redundant date column
merged_test_data.drop(columns=['h_date'], inplace=True)
merged_test_data.head()

,counter_name,site_name,bike_count,date,latitude,longitude,log_bike_count,hour,is_holiday
0,28 boulevard Diderot E-O,28 boulevard Diderot,1.0,2021-08-10,48.846028,2.375429,0.693147,5,0
1,28 boulevard Diderot E-O,28 boulevard Diderot,2.0,2021-08-10,48.846028,2.375429,1.098612,6,0
2,28 boulevard Diderot E-O,28 boulevard Diderot,1.0,2021-08-10,48.846028,2.375429,0.693147,7,0
3,28 boulevard Diderot E-O,28 boulevard Diderot,0.0,2021-08-10,48.846028,2.375429,0.000000,9,0
4,28 boulevard Diderot E-O,28 boulevard Diderot,1.0,2021-08-10,48.846028,2.375429,0.693147,10,0


5. Do the regression

In [162]:
def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    X['date'] = pd.to_datetime(X['date'])
    X.loc[:, "year"] = X["date"].dt.year
    X.loc[:, "month"] = X["date"].dt.month
    X.loc[:, "day"] = X["date"].dt.day
    X.loc[:, "weekday"] = X["date"].dt.weekday
    return X.drop(columns=["date"])

merged_train_data = _encode_dates(merged_train_data)
merged_test_data = _encode_dates(merged_test_data)
merged_train_data.head()

,counter_name,site_name,bike_count,latitude,longitude,log_bike_count,hour,is_holiday,year,month,day,weekday
0,28 boulevard Diderot E-O,28 boulevard Diderot,0.0,48.846028,2.375429,0.000000,2,0,2020,9,1,1
1,28 boulevard Diderot E-O,28 boulevard Diderot,1.0,48.846028,2.375429,0.693147,3,0,2020,9,1,1
2,28 boulevard Diderot E-O,28 boulevard Diderot,0.0,48.846028,2.375429,0.000000,4,0,2020,9,1,1
3,28 boulevard Diderot E-O,28 boulevard Diderot,4.0,48.846028,2.375429,1.609438,15,0,2020,9,1,1
4,28 boulevard Diderot E-O,28 boulevard Diderot,9.0,48.846028,2.375429,2.302585,18,0,2020,9,1,1


In [165]:
X_merged_train_data = merged_train_data.drop(columns=['bike_count','log_bike_count'])
Y_merged_train_data = merged_train_data['log_bike_count']

In [163]:
# select some potentially relevant features for prediction
selected_features = ['latitude', 'longitude', 'hour', 'is_holiday', 'year', 'month', 'day', 'weekday'] 

# Preprocessing pipeline for standardization
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), selected_features)
    ])

# Combine preprocessing and model training in a pipeline
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

# define number of splits
tscv = TimeSeriesSplit(n_splits=5)

In [167]:
# Cross-validation scores
cross_val_scores = cross_val_score(model, X_merged_train_data, Y_merged_train_data, cv=tscv, scoring='neg_root_mean_squared_error')

# Average RMSE
avg_rmse = -np.mean(cross_val_scores)
print(f"Average RMSE: {avg_rmse}")

Average RMSE: 1.6069383378226643


### adding weather data

In [36]:
external_data_df = pd.read_csv('external_data.csv')

In [37]:
# Drop columns with 0 non-null values from external_data_df
cols_to_drop = [col for col in external_data_df.columns if external_data_df[col].notnull().sum() == 0]
external_data_df.drop(cols_to_drop, axis=1, inplace=True)

# Convert 'date' columns to datetime
external_data_df['date'] = pd.to_datetime(external_data_df['date'])

In [39]:
# Merge the data on the 'date' column
merged_df = pd.merge(merged_train_data, external_data_df, on='date', how='inner')

# Calculate the median for each column and replace NaN values
medians = merged_df.median(numeric_only=True)
merged_df.fillna(medians, inplace=True)


# Drop the original categorical columns
merged_df.drop(['counter_name', 'site_name'], axis=1, inplace=True)

# Split the data into features and target
y = merged_df['log_bike_count']
X = merged_df.drop('log_bike_count', axis=1)

In [40]:
merged_df.head()

,bike_count,date,latitude,longitude,log_bike_count,hour,is_holiday,numer_sta,pmer,tend,...,hnuage1,nnuage2,ctype2,hnuage2,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4
0,0.0,2020-09-01,48.846028,2.375429,0.000000,2,0,7149,102050,-10,...,1170.0,6.0,6.0,1620.0,7.0,6.0,1920.0,4.0,8.0,3600.0
1,1.0,2020-09-01,48.846028,2.375429,0.693147,3,0,7149,102050,-10,...,1170.0,6.0,6.0,1620.0,7.0,6.0,1920.0,4.0,8.0,3600.0
2,0.0,2020-09-01,48.846028,2.375429,0.000000,4,0,7149,102050,-10,...,1170.0,6.0,6.0,1620.0,7.0,6.0,1920.0,4.0,8.0,3600.0
3,4.0,2020-09-01,48.846028,2.375429,1.609438,15,0,7149,102050,-10,...,1170.0,6.0,6.0,1620.0,7.0,6.0,1920.0,4.0,8.0,3600.0
4,9.0,2020-09-01,48.846028,2.375429,2.302585,18,0,7149,102050,-10,...,1170.0,6.0,6.0,1620.0,7.0,6.0,1920.0,4.0,8.0,3600.0


In [43]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print the RMSE
print(f"Root Mean Squared Error (RMSE): {rmse}")


DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'nu